# Predykcja cen nieruchomości oparta na modelu maszyny wektorów nośnych

## Wstęp

Celem niniejszego projektu będzie zaimplemetowanie trzech regresorów służących do predykcji ceny domu, opartych na modelu maszyny wektorów nośnych (SVM), bazujących na kernelu:
1. liniowym
2. wielomianowym
3. RBF. 

Dokumentacja będzie skupiała się na opisie rozwiązania problemu przewidywania cen nieruchomości z perspektywy potencjalnego kupującego oraz sprzedającego. Zebrane dane dotyczą obszarów położonych wokół amerykańskiego miasta Boston i pochodzą z 1974 roku. 

Model przewidujący cene rynkową nieruchomości mógłby być znakomitym narzędziem w rękach agenta, którego praca opiera się na analizach wartości domów. Z drugiej strony, potencjalny nabywca również mógłby wykorzystać model, aby zorientować się, w jakim przedziale cenowym nieruchomość, którą jest zainteresowany, się znajduje.

Na cene domu, oprócz takich cech jak powierzchnia użytkowa i lokalizacja, wpływ ma wiele mniej oczywistych cech. W projektowanym modelu zostanie podjęta próba wykorzystania wszystkich użytecznych informacji w celu przeprowadzenia pełnej analizy i oszacowaniu cen nieruchomości.

Cały etap projektowania modelu podzielony został na etapy, które umożliwią zoptymalizowanie danych pod kątem logicznym, tak, aby możliwe stało się ich jak najlepsze wykorzystanie w celach predykcji. Tymi krokami są:

1. Zaimportowanie danych oraz bibliotek
2. Analiza cechy predykowanej - ceny nieruchomości
3. Analiza charakterystyk
4. Imputacja brakujących danych oraz wyczyszczenie danych
5. Optymalizacja danych
6. Modelowanie i predykcja

# 1. Zaimportowanie danych oraz bibliotek

## 1.1 Zaimportowanie bibliotek

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
from scipy.stats import skew
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import norm
from collections import Counter
from sklearn.linear_model import LinearRegression,LassoCV, Ridge, LassoLarsCV,ElasticNetCV
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler
warnings.filterwarnings('ignore')
sns.set(style='white', context='notebook', palette='deep')
%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

Na początku zaimportowane zostaną wszystkie biblioteki, dzięki którym znaczącą ułatwiony zostanie proces analizowania danych. Wśród nich znajdują się biblioteki:

1. Pandas - jest opensource'ową biblioteką, wydawaną na licencji BSD, która dostarcza zaawansowanych mechanizmów tworzenia struktur danych oraz ich analizy w języku Python;
2. NumPy - jest podstawową bilblioteką wykorzystywaną w obliczeniach inżynierskich wykonywanych w Pythonie. Dostarcza mechanizmów tworzenia N-wymiarowych tablic, skomplikowanych funkcji matematycznych oraz integracji z kodami źródłowymi napisanymi w językach C/C++ oraz Fortran. Jednak najważniejszą funkcjonalnością pozostaje możliwość tworzenia kontenerów danych w celach ich późniejszej analizy.
3. Seaborn - jest biblioteką, opartą o pakiet matplotlib, umożliwiającą wizualizacje danych i analiz w Pythonie na zaawansowanym poziomie
4. matplotlib - ta biblioteka dostarcza mechanizmów rysowania gotowych do publikacji wykresów w wielu ogólnodostępnych formatach oraz w wielu środowiskach, w tym w wykorzystywanym w projekcie Jupyter Notebook.
6. SciPy - dostarcza wielu mechanizmów, które umożliwiają na wykonanie często używanych operacji numerycznych, jak integracja, interpolacja, optymalizacja czy statystyka
7. scikit-learn - jedna z najważniejszych bibliotek, która dostarcza mechanizmów sztucznej inteligencji w języku Python. Zawiera implmenetacje alogrytmów klasyfikacji, regresji oraz klasteryzacji opartych na metodach gradientowego wzmacniania regrecji, lasów losowych czy wykorzystywanych w projekcie maszynach wektorów nośnych. Dodatkowo jest kompatybilna i współpracuje razem z bibliotekami SciPy oraz NumPy

## 1.2 Zaimportowanie danych

In [ ]:
# Load train and Test set
train = pd.read_csv("./boston-housing/train.csv")
test = pd.read_csv("./boston-housing/test.csv")

W dwóch plikach .csv zawarte są dane, które zostaną wykorzystane w celach budowy modeli regresorów. Dane pochodzą z repoztorium uczenia maszynowego UCI. Zostały zebrane w roku 1978 i zawierają 506 rekordów przedstawiających zagregowane informacje o nieruchomości oraz związanych z nią cechach.

In [ ]:
# Check the numbers of samples and features
print("The train data size before dropping Id feature is : {} ".format(train.shape))
print("The test data size before dropping Id feature is : {} ".format(test.shape))

# Save the 'Id' column
train_ID = train['ID']
test_ID = test['ID']

# Now drop the 'Id' column since it's unnecessary for the prediction process.
train.drop("ID", axis = 1, inplace = True)
test.drop("ID", axis = 1, inplace = True)

# Check data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test.shape))

Widzimy, że dane z pliku train.csv zawierają jedną kolumne więcej niż dane z pliku test.csv. Tą kolumną jest cena nieruchomości, którą model ma za zadanie przewidzieć.

In [ ]:
train.head()

In [ ]:
test.head()

Cechami, które opisują każdą z nieruchomości w zaimportowanych danych są:
1. crim - współczynnik przestępstw per capita w danym mieście
2. zn - proporcja strefy gruntów mieszkalnych z mieszkaniami powyżej 25.000 stóp kwadratowych
3. indus - proporcja działalności niehandlowwej w danym mieście
4. chas - cecha określająca przyleganie nieruchomości do rzeki Charles (= 1 gdy przylega do rzeki, 0 w przeciwnym wypadku
5. nox - koncentracja tlenku azotu (w cząsteczkach na 10 milionów)
6. rm - średnia liczba pokoi na nieruchomość
7. age - proporcja nieruchomości zajętych wybudowanych po 1940 roku
7. dis - średnia ważona odległości od 5 największych bostońskich centrów zatrudnienia
8. rad - index określający dostępność nieruchomości do autostrad
9. tax - stopa podatku od nieruchomości per 10.000 dolarów
10. ptratio - stosunek liczby uczniów do nauczycieli w danym mieście
11. black - 1000(Bk - 0.63)^2 gdzie Bk to stosunek czarnoskórych w danym mieście
12. lstat - procent populacji żyjący poniżej granicy ubóstwa
13. medv - średnia wartość nieruchomości w 1000 dolarów

# Analiza cechy predykowanej - ceny nieruchomości 

In [ ]:
# Getting Description
train['medv'].describe()

In [ ]:
# Plot Histogram
sns.distplot(train['medv'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['medv'])
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Median Value distribution')

fig = plt.figure()
res = stats.probplot(train['medv'], plot=plt)
plt.show()

print("Skewness: %f" % train['medv'].skew())
print("Kurtosis: %f" % train['medv'].kurt())

# 3. Multivariable Analysis

In [ ]:
# Correlation Matrix Heatmap
corrmat = train.corr()
f, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(corrmat, vmin=-1, vmax=1, square=True, center=0, annot=True, fmt='.2f');

In [ ]:
cols = corrmat.abs().nlargest(14, 'medv')['medv'].index
most_corr = pd.DataFrame(cols)
most_corr.columns = ['Most Correlated Features']
most_corr

**BLACKS**

In [ ]:
sns.jointplot(x=train['black'], y=train['medv'], kind='reg');

In [ ]:
var = 'black'
data = pd.concat([train['medv'], train[var]], axis=1)
f, ax = plt.subplots(figsize=(25, 12))
fig = sns.boxplot(x=var, y="medv", data=data)
fig.axis(ymin=0, ymax=52);

**F A T A L N E** dane 

In [ ]:
# usuwamy je
train.drop(['black'], axis=1, inplace=True)
test.drop(['black'], axis=1, inplace=True)

In [ ]:
# usuwamy też rzekę - też lipna
train.drop(['chas'], axis=1, inplace=True)
test.drop(['chas'], axis=1, inplace=True)

**Dane o najwyższej wartości bezwzględnej korelacji**

In [ ]:
sns.jointplot(x=train['lstat'], y=train['medv'], kind='reg');
print(train.shape[0])

In [ ]:
# Removing outliers manually 
train = train.drop(train[(train['medv']>49.99) & (train['lstat']>8)].index).reset_index(drop=True)

In [ ]:
sns.jointplot(x=train['lstat'], y=train['medv'], kind='reg');
print(pearsonr(train['lstat'], train['medv']));
print(train.shape[0]);

In [ ]:
sns.jointplot(x=train['rm'], y=train['medv'], kind='reg');

In [ ]:
# wiekość pokoi może być różna - nie odrzucamy tu obserwacji odstających
# train = train.drop(train[(train['medv'] > 8 * train['rm'] - 8)].index).reset_index(drop=True)

In [ ]:
# sns.jointplot(x=train['rm'], y=train['medv'], kind='reg');
# print(pearsonr(train['rm'], train['medv']));
# print(train.shape[0]);

In [ ]:
sns.jointplot(x=train['ptratio'], y=train['medv'], kind='reg');
# w jednym mieście może być wiele regionów o różnej charakterystyce; 
# współczynnik ptratio jest wspólny dla wszystkich regionów miasta - biednych i bogatych
# jednak można zauważyć silną ujemną korelację miedzy tymi wielkościami
# ciężko cokolwiek odrzucić

In [ ]:
var = 'ptratio'
data = pd.concat([train['medv'], train[var]], axis=1)
f, ax = plt.subplots(figsize=(25, 12))
fig = sns.boxplot(x=var, y="medv", data=data)
fig.axis(ymin=0, ymax=52);

In [ ]:
sns.jointplot(x=train['indus'], y=train['medv'], kind='reg');

In [ ]:
var = 'indus'
data = pd.concat([train['medv'], train[var]], axis=1)
f, ax = plt.subplots(figsize=(25, 12))
fig = sns.boxplot(x=var, y="medv", data=data)
fig.axis(ymin=0, ymax=52);

In [ ]:
sns.jointplot(x=train['tax'], y=train['medv'], kind='reg');

In [ ]:
var = 'tax'
data = pd.concat([train['medv'], train[var]], axis=1)
f, ax = plt.subplots(figsize=(25, 12))
fig = sns.boxplot(x=var, y="medv", data=data)
fig.axis(ymin=0, ymax=52);

# Uzupełnianie danych - nasze dane są kompletne

In [ ]:
# We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
train["medv"] = np.log1p(train["medv"])

#Check the new distribution 
sns.distplot(train['medv'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['medv'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

fig = plt.figure()
res = stats.probplot(train['medv'], plot=plt)
plt.show()

y_train = train.medv.values

print("Skewness: %f" % train['medv'].skew())
print("Kurtosis: %f" % train['medv'].kurt())

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['medv'], axis=1, inplace=True)
print("Train data size is : {}".format(train.shape))
print("Test data size is : {}".format(test.shape))
print("Combined dataset size is : {}".format(all_data.shape))

In [ ]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: (skew(x.dropna()))).sort_values(ascending=False)
skewness = pd.DataFrame({'Positive skewed Features' :skewed_feats})
skewness.head(11)

In [ ]:
# skewness = skewness[skewness > 0.8]
# print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
skewness = skewness[skewness > 0.8]
skewness = skewness.dropna()

from scipy.special import boxcox1p

skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    all_data[feat] = boxcox1p(all_data[feat], lam)
    all_data[feat] += 1

In [ ]:
# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: (skew(x.dropna()))).sort_values(ascending=False)
skewness = pd.DataFrame({'Positive skewed Features' :skewed_feats})
skewness.head(11)

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]